In [229]:
import requests
from bs4 import BeautifulSoup

import pandas as pd
import re

## getting air force base locations

In [310]:
airforce_wiki_url = "https://en.wikipedia.org/wiki/List_of_United_States_Air_Force_installations"
df_usaf = pd.read_html(airforce_wiki_url)[1]

In [350]:
df_usaf.head(3)

,Name,Location,State,Coordinates,Commanding organization,Wing or unit emblem,Host wing or primary unit,Primary missions and units,lat,lon
0,Altus Air Force Base,Altus,Oklahoma,34°39′59″N 099°16′05″W﻿ / ﻿34.66639°N 99.26806°W,Air Education and Training Command,NaN,97th Air Mobility Wing,The 97th Air Mobility Wing trains crews to ope...,34.66639,-99.26806
1,Joint Base Anacostia-Bolling,Southwest,"Washington, D.C.",38°50′34″N 077°00′58″W﻿ / ﻿38.84278°N 77.01611°W,Air Force District of Washington,NaN,11th Wing,"US Navy operated joint base, accommodating Geo...",38.84278,-77.01611
2,Joint Base Andrews-Naval Air Facility Washington,Camp Springs,Maryland,38°48′39″N 076°52′01″W﻿ / ﻿38.81083°N 76.86694°W,Air Force District of Washington,NaN,11th Wing,USAF operated joint base. The 11th Wing provid...,38.81083,-76.86694


In [312]:
def extract_coords(s):
    # extract coordinates from the Coordinates column in the table
    s = s.split('/')[1]
    # remove BOM from beginning of string
    s = s.replace('\ufeff', '')
    s = s.strip()
    # remove all chars except decimals and NSEW letter
    coords = ''.join(item for item in s if item in ' .1234567890NSEW').split()
    # remove NSEW designation from each coord, convert to float, and assign negative if needed
    coords = [float(coord[:-1]) if coord[-1] in 'NE' else -float(coord[:-1]) for coord in coords]
    return coords

In [319]:
coords = pd.DataFrame(df_usaf['Coordinates'].apply(extract_coords).to_list(), columns = ['lat', 'lon'])

In [320]:
df_usaf = df_usaf.join(coords)

In [349]:
df_usaf.head(3)

,Name,Location,State,Coordinates,Commanding organization,Wing or unit emblem,Host wing or primary unit,Primary missions and units,lat,lon
0,Altus Air Force Base,Altus,Oklahoma,34°39′59″N 099°16′05″W﻿ / ﻿34.66639°N 99.26806°W,Air Education and Training Command,NaN,97th Air Mobility Wing,The 97th Air Mobility Wing trains crews to ope...,34.66639,-99.26806
1,Joint Base Anacostia-Bolling,Southwest,"Washington, D.C.",38°50′34″N 077°00′58″W﻿ / ﻿38.84278°N 77.01611°W,Air Force District of Washington,NaN,11th Wing,"US Navy operated joint base, accommodating Geo...",38.84278,-77.01611
2,Joint Base Andrews-Naval Air Facility Washington,Camp Springs,Maryland,38°48′39″N 076°52′01″W﻿ / ﻿38.81083°N 76.86694°W,Air Force District of Washington,NaN,11th Wing,USAF operated joint base. The 11th Wing provid...,38.81083,-76.86694


## getting postal office locations

In [174]:
from io import StringIO

In [252]:
def gather_USPS_data():

    # create session and specify header due to cookies issue with the about.usps.com site
    s = requests.Session()
    headers = {
        'Accept-Encoding': 'gzip, deflate, sdch',
        'Accept-Language': 'en-US,en;q=0.8',
        'Upgrade-Insecure-Requests': '1',
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Cache-Control': 'max-age=0',
        'Connection': 'keep-alive',
    }
    r = s.get("https://about.usps.com/who/legal/foia/owned-facilities.htm", headers = headers)
    l = soup.find_all('ul', class_='list-unstyled list-multi-column')[0].find_all('li')

    # for each state csv in the html table, create a dataframe and append to list to be concat after
    dfs = []
    for item in l:
        # create path using html 
        path = "https://about.usps.com" + item.find('a').get('href')
        r_csv = s.get(path, headers = headers)
        # remove some misc text from the beginning of the string 
        r_csv = r_csv.text.replace("""Owned Area - Building Inventory 2,,,,,,,,,,,,,,,,,,,,\r\n,,,,,,,,,,,,,,,,,,,,\r\n,,,,,,,,,,,,,,,,,,,,\r\n""", '')
        st = StringIO(r_csv)
        # drop the last two rows, which are misc 
        df = pd.read_csv(st)[:-2]
        dfs.append(df)

    final_df = pd.concat(dfs, ignore_index = True)
    return final_df

In [253]:
df_usps = gather_USPS_data()

In [348]:
df_usps.head(3)

,District,Fin-Sub,Chrgbl Fin No,PO Name,Unit Name,Property Address,County,City,ST,ZIP Code,...,Ownership,FDB ID (All),AMS Locale Key (All),FDB Facility Type (All),FDB Facility Subtype (All),Building Ownership Description,Land Desc,Space Certified Indicator,Bldg Occu Date,Int Sq Ft
0,Alabama,010120-G02,10120.0,ALBERTVILLE,MAIN OFFICE,210 S HAMBRICK ST,MARSHALL,ALBERTVILLE,AL,35950-1624,...,Owned,1352597.0,Y10022,Post Office,Main Post Office,"USPS Building, Const. by USPS","USPS Land, Not Prev. Leased",No,12/1/1983,"8,913"
1,Alabama,010150-G03,10150.0,ALEXANDER CITY,MAIN OFFICE,233 LEE ST,TALLAPOOSA,ALEXANDER CITY,AL,35010-2654,...,Owned,1352654.0,Y10026,Post Office,Administrative Post Office (APO),"USPS Building, Const. by USPS","USPS Land, Not Prev. Leased",Yes,9/1/1984,"7,748"
2,Alabama,010240-G01,10240.0,ALTON,MAIN OFFICE MODULAR,5548 JOHNSON ST,JEFFERSON,ALTON,AL,35015-2001,...,Owned,1352868.0,Y10035,Post Office,Remotely Managed Post Office (RMPO),"USPS Building, Not Prev. Leased",Land Data on separate record,No,9/1/1995,672


### write out chunks of df_usps no larger than 10k records each

needs example format:

1,4600 Silver Hill Rd,Suitland,MD,20746  
2,436 15th St SE, Washington, DC,20003  
  


key columns are Street, City, State, Zip

For geocoding purposes, make sure there are no duplicates on the key columns in order to reduce queries


In [336]:
df_usps[['Property Address', 'City', 'ST', 'ZIP Code']].duplicated().value_counts()

True     10383
False     8343
dtype: int64

thats a ton of dupes ... locations might have more than one USPS-role and therefore show up multiple times

In [339]:
uniques = df_usps[['Property Address', 'City', 'ST', 'ZIP Code']].drop_duplicates()

In [340]:
uniques

,Property Address,City,ST,ZIP Code
0,210 S HAMBRICK ST,ALBERTVILLE,AL,35950-1624
1,233 LEE ST,ALEXANDER CITY,AL,35010-2654
2,5548 JOHNSON ST,ALTON,AL,35015-2001
3,520 E THREE NOTCH ST,ANDALUSIA,AL,36420-3128
4,7312 HIGHWAY 207,ANDERSON,AL,35610-4840
...,...,...,...,...
17968,US HIGHWAY 14N,YELLOWSTONE NATIONAL PARK,WY,82190-9998
17969,US HIGHWAY 89/191/287,YELLOWSTONE NATIONAL PARK,WY,82190-9998
17970,US HIGHWAY 14/16 and US HIGHWAY 89-191,YELLOWSTONE NATIONAL PARK,WY,82190-9998
17971,1000 MAMMOTH,YELLOWSTONE NATIONAL PARK,WY,82190-9650


write out in chunks just in case the # of uniques is > 10000

In [341]:
chunk_size = 10000

for i in range(round(len(uniques)/chunk_size)):
    chunk = uniques[i*chunk_size:chunk_size*(i+1)]
    chunk.to_csv("data/uscensus/usps_chunk_{}.csv".format(i), header = False)

In [342]:
# pass to geocoding service
# https://geocoding.geo.census.gov/geocoder/Geocoding_Services_API.pdf
!curl --form addressFile=@data/uscensus/usps_chunk_0.csv --form benchmark=Public_AR_Current https://geocoding.geo.census.gov/geocoder/locations/addressbatch --output data/uscensus/geocoderesult1.csv

% Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1318k  100  949k  100  369k   2240    871  0:07:14  0:07:13  0:00:01  153k


In [351]:
df_geocoderesult = pd.read_csv("data/uscensus/geocoderesult.csv", names = ['index', 'input', 'match', 'match_type', 'output', 'coords', 'tigerlineID', 'side'])

In [353]:
df_geocoderesult.head()

,index,input,match,match_type,output,coords,idk1,idk2
0,17288,"22433 RANDOLPH DR, DULLES, VA, 20104-9998",Match,Non_Exact,"22433 Randolph Dr, DULLES, VA, 20103","-77.45221,38.997852",62362877.0,R
1,17289,"44715 PRENTICE DR, DULLES, VA, 20101-9998",Match,Non_Exact,"44715 Prentice Dr, DULLES, VA, 20166","-77.4586,39.000824",62362881.0,L
2,17284,"10001 COUNTY DR, DISPUTANTA, VA, 23842-9998",Match,Exact,"10001 County Dr, DISPUTANTA, VA, 23842","-77.22661,37.124454",54181994.0,L
3,17285,"22365 DREWRY RD, DREWRYVILLE, VA, 23844-9998",Match,Exact,"22365 Drewry Rd, DREWRYVILLE, VA, 23844","-77.30633,36.71563",82709745.0,R
4,4970,"345 E SUNSET BLVD, GERLACH, NV, 89412-9800",Tie,NaN,NaN,NaN,NaN,NaN


In [357]:
df_geocoderesult[df_geocoderesult.coords.isna()]

,index,input,match,match_type,output,coords,idk1,idk2
4,4970,"345 E SUNSET BLVD, GERLACH, NV, 89412-9800",Tie,NaN,NaN,NaN,NaN,NaN
5,17286,"1 TOWN CENTER DR, DUBLIN, VA, 24084-9998",No_Match,NaN,NaN,NaN,NaN,NaN
8,17287,"60 DOUBLE CABIN RD, DUGSPUR, VA, 24325-9998",No_Match,NaN,NaN,NaN,NaN,NaN
11,17282,"105 TEAL CT, DANVILLE, VA, 24541-9998",No_Match,NaN,NaN,NaN,NaN,NaN
15,4969,"2244 FOOTHILL RD, GENOA, NV, 89411-9998",No_Match,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
8332,4950,"134 E 6TH ST, WAHOO, NE, 68066-9998",Tie,NaN,NaN,NaN,NaN,NaN
8333,4951,"1500 S 112TH ST, WALTON, NE, 68461-9538",No_Match,NaN,NaN,NaN,NaN,NaN
8334,4952,"120 N PEARL ST, WAYNE, NE, 68787-1932",Tie,NaN,NaN,NaN,NaN,NaN
8337,3629,"130 N GREENE ST, BALTIMORE, MD, 21201-9997",No_Match,NaN,NaN,NaN,NaN,NaN
